# 1. Understand the dataset

- Objective of the dataset: the dataset consist of movies, users & activities from letterboxd.com, used to predict the next movies users would watch

- The dataset consists of 3 important components:
    - `movies`: Contain the meta information for movies, it also comes with other external properties like crews, languages,...
    - `users`: Represent an user
    - `users_and_movies`: Contain user activities in relation to movies, it consist of 

# 2. Import data

> The dataset is a SQLite file, we will use [polars](https://pola.rs/) for working with the dataset

- Load libraries

In [ ]:
import sqlite3
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

- Opening database

In [ ]:
conn = sqlite3.connect("/home/leminhohoho/repos/movie-lens/db/letterboxd_2.db")

- Load tables

In [ ]:
users = pl.read_database("SELECT * FROM users", connection=conn)
movies = pl.read_database("SELECT * FROM movies", connection=conn)
crews = pl.read_database("SELECT * FROM crews", connection=conn)
crews_and_movies = pl.read_database("SELECT * FROM crews_and_movies", connection=conn)
genres = pl.read_database("SELECT * FROM genres", connection=conn)
genres_and_movies = pl.read_database("SELECT * FROM genres_and_movies", connection=conn)
themes = pl.read_database("SELECT * FROM themes", connection=conn)
themes_and_movies = pl.read_database("SELECT * FROM themes_and_movies", connection=conn)
releases = pl.read_database("SELECT * FROM releases", connection=conn)
studios = pl.read_database("SELECT * FROM studios", connection=conn)
studios_and_movies = pl.read_database("SELECT * FROM studios_and_movies", connection=conn)
countries_and_movies = pl.read_database("SELECT * FROM countries_and_movies", connection=conn)
languages_and_movies = pl.read_database("SELECT * FROM languages_and_movies", connection=conn)
users_and_movies = pl.read_database("SELECT * FROM users_and_movies", connection=conn)

with pl.Config(tbl_cols=-1):
    print(users)
    print(movies)
    print(crews)
    print(crews_and_movies)
    print(genres)
    print(genres_and_movies)
    print(themes)
    print(themes_and_movies)
    print(releases)
    print(studios)
    print(studios_and_movies)
    print(countries_and_movies)
    print(languages_and_movies)
    print(users_and_movies)

# 3. Visualization

- Comparison of movies made between studios

In [ ]:
studios_movies_count = studios_and_movies.join(
    studios, left_on="studio_id", right_on="id"
).with_columns(
    pl.col("name").alias("studio_name")
).drop("name").join(
    movies, left_on="movie_id", right_on="id" 
).select(
    pl.col("studio_name"),
    pl.col("name"),
    pl.col("movie_id"),
).group_by("studio_name").agg(pl.col("movie_id").count().alias("movies_made")).sort(pl.col("movies_made"), descending=True).head(20)

print(studios_movies_count)

plt.figure(figsize=(20, 12))
plt.bar(studios_movies_count['studio_name'].to_list(), studios_movies_count['movies_made'].to_list(), color='skyblue')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Studio')
plt.ylabel('Number of Movies Made')
plt.title('Number of Movies Made by Each Studio (top )')
plt.tight_layout()
plt.savefig("/home/leminhohoho/repos/movie-lens/ml/data/studios_movies_made_comparison.png")
plt.show()

- Genres popularity over time

In [ ]:
genres_with_time = genres_and_movies.join(
    movies.drop("name"), left_on="movie_id", right_on="id" 
).join(
    genres, left_on="genre_id", right_on="id" 
).join(
    releases.filter(pl.col("release_type") == "Premiere"), left_on="movie_id", right_on="movie_id" 
).with_columns(
    pl.col("date").str.strptime(pl.Date, format="%d %b %Y").alias("date")
).with_columns(
    pl.col("date").dt.year().alias("year"),
).select(
    pl.col("name"), pl.col("year")
).group_by(["year", "name"]).count().sort(pl.col("year"))

with pl.Config(tbl_cols=-1):
    print(genres_with_time)

df_pivot = genres_with_time.pivot(
    values="count",
    index="year",
    columns="name"
).sort("year")

plt.figure(figsize=(24, 12))
for genre in df_pivot.columns[1:]:
    plt.plot(df_pivot["year"], df_pivot[genre], label=genre)

plt.xlabel("Year")
plt.ylabel("Number of Movies")
plt.title("Movie Count per Genre Over Years")
plt.legend()
plt.grid(True)
plt.savefig("/home/leminhohoho/repos/movie-lens/ml/data/movies_count_based_on_genres_over_years.png")
plt.show()

In [ ]:
rating_per_genre = (
    genres_and_movies
    .join(users_and_movies, left_on="movie_id", right_on="movie_id")
    .join(genres, left_on="genre_id", right_on="id")
    .group_by(pl.col("name"))
    .agg(pl.col("rating").mean().alias("avg_rating"))
    .select(["name", "avg_rating"])
)

# Prepare data for plotting
genre_names = rating_per_genre["name"].to_list()
genre_ratings = rating_per_genre["avg_rating"].to_list()

# Normalize ratings for colormap
rating_array = np.array(genre_ratings)
norm_ratings = (rating_array - rating_array.min()) / (rating_array.max() - rating_array.min() + 1e-8)
colors = plt.cm.viridis(norm_ratings)

# Close previous plots
plt.close("all")

# Create plot
fig, ax = plt.subplots(figsize=(24, 12))
bars = ax.bar(genre_names, genre_ratings, color=colors, edgecolor='black', alpha=0.8)

# Customize axes
ax.set_xlabel('Genres', fontsize=20)
ax.set_ylabel('Average Rating', fontsize=20)
ax.set_title('Average Rating by Genre', fontsize=20, fontweight='bold')
plt.xticks(rotation=45, ha='right', fontsize=20)

# Increase y-limit slightly so labels are above bars
ax.set_ylim(0, max(genre_ratings) * 1.1)

# Add value labels above bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + max(genre_ratings)*0.02,  # small offset above bar
            f'{height:.2f}', ha='center', va='bottom', fontsize=20, fontweight='bold')

# Styling
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("/home/leminhohoho/repos/movie-lens/ml/data/rating_per_genre.png")
plt.show()